# NAS for CNN

## Structural elements

- Branches
- Connections
- Layers
    - Input
        - Input shape (first input is standard, all others are calculated)
        - Batch size 
    - 2D convolutional
        - Filters
        - Kernel size
        - Padding
        - Activation
    - Pooling
        - Max
            - Size
            - Strides
            - Padding
        - Average
            - Size
            - Strides
            - Padding
    - Dropout
        - Rate
    - Batch normalisation
    - Concatenate
        - Axis
    - Flatten
    - Dense
        - Units (1, 2)
        - Activation (sigmoid, softmax)
    - Global pooling
        - Max
        - Average

## Genetic elements

- GA
    - Type
    - Number of generations
- Population
    - Size
    - Encoding type
    - Individual size
        - Variable or fixed
- Fitness
- Selection
- Reproduction
- Mutation

In [57]:
import numpy as np
import tensorflow as tf
from scipy.spatial import distance
from tensorflow.keras import backend
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPool2D, AveragePooling2D, Conv2D, Input, SpatialDropout2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras import backend as K
from IPython.display import clear_output

In [2]:
params = {
    'input':{
        'batch':[8, 16, 32] # input batch size
        },
    'conv':{
        'kernel':[1, 3, 5, 7], 
        'filter':[2, 4, 8, 16],
        'padding':['valid', 'same'],
        'activation':['tanh', 'relu', 'selu', 'elu']
        },
    'pool':{
        'type':['max', 'average'],
        'size':[2, 3, 4, 5],
        'padding':['valid', 'same']
        },
    'dropout':{
        'type':['dropout','spatial2D'],
        'rate':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        },
    'output':{
        'type':['global', 'dense']
        }
}
    

In [3]:
individual = [
    # Input layer
    'batch_size_index', 
    
    # Convolutional layer
    'kernel_index', 'filter_index', 'padding_index', 'activation_index',

    # Pooling layer
    'type_index', 'size_index', 'padding_index',

    # Dropout layer
    'type_index', 'rate_index',

    # Output layer
    'type_index',

    # Add layer
    ## Layer/Branch 1
    'indicator', 'branch_indicator', 'type', 'from', 'to',

    ## Layer/Branch 2
    'indicator', 'branch_indicator', 'type', 'from', 'to',

    ## ... variable length array

    ## Layer/Branch n
    'indicator', 'branch_indicator', 'type', 'from', 'to'


    ]
    


In [4]:
def get_state_space(parameters):

    """
    Get the encoding length of an individual based on the input parameters dictionary
    """

    len_lists = []
    len_values = [] # stores the maximum lengths of bits required to define the length of the lists in the parameters dict
    # max_ind = []
    # bit_max_int = []
    # bit_max = ''

    for layer in params.keys():

            for value_list in list(params[layer].values()):

                # max_ind.append(len(value_list) - 1)

                # bit_max = bit_max + bin(len(value_list) - 1)[2:]

                len_values.append(len(bin(len(value_list) - 1)[2:])) 

                len_lists.append(len(value_list))
        

    gene_length = np.max(len_values)
    
    individual_length = len(len_lists) * gene_length

    # len_values = np.array(len_values)

    len_lists = np.array(len_lists)

    return individual_length, gene_length, len_lists

In [5]:
ind_len, gene_len, len_list = get_state_space(params)
print(ind_len, gene_len, len_list)

44 4 [3 4 4 2 4 2 4 2 2 9 2]


In [6]:
def generate_population(length, parameter_len, n = 10, seed = None):
    """
    Generate population given the number of individuals in a population and the the required binary length
    """

    bit_length = int(length/len(parameter_len))

    if seed != None:

        np.random.seed(seed)

    population = np.zeros(shape=(n, length), dtype=int)
    
    for i in range(n):
        
        bits = ''
        for l in parameter_len:

            choice = np.random.choice(np.arange(0, l))

            bit_choice = bin(choice)[2:]

            bits += ('0' * (bit_length - len(bit_choice))) + bit_choice

        population[i] = np.array([int(x) for x in bits])
    
    return population


In [7]:
pop = generate_population(ind_len, len_list, n = 100, seed = 43)
print(pop[:2])

[[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1
  0 0 0 1 0 0 0 1]]


In [8]:
def decoder(chromosome, gene_length):

    """
    Converts the binary bitstring vector to integer phenotype and performs check to see if it is valid with respect to the constraints
    """

    chromo_length = len(chromosome)

    start_ind = np.arange(0, chromo_length, gene_length)

    phenotype = np.array([int(str(''.join(map(str, chromosome[x:x + gene_length]))), 2) for x in start_ind])

    # is_valid = np.any(phenotype < parameter_lengths - 1)

    return phenotype #, is_valid


In [9]:
pheno = decoder(pop[0], gene_len)
print(pheno)

[0 0 3 1 1 0 0 1 1 2 0]


In [10]:
def validity_check(phenotype, parameter_lengths):
    return np.any(phenotype < parameter_lengths - 1)

In [11]:
valid = validity_check(pheno, len_list)
print(valid)

True


In [12]:
def hamming_distance(population, selection_probability = 0.1, seed = None):

    n = population.shape[0]

    n_select = int(np.ceil(n*selection_probability))

    mean_hamming = 0

    if seed != None:
        np.random.seed(seed)

    for individual in np.random.choice(np.arange(0, n, n_select), n_select):

        mean_hamming += np.mean([distance.hamming(population[individual], population[x]) for x in range(n)])

    return mean_hamming/n_select

In [13]:
hamming_distance(pop)

0.20345454545454547

In [14]:
def fitness(fitness_input, population = None, model_params = None, age = None, beta = 10):

    diversity = 1

    n_params = 1

    if population != None:
        diversity = hamming_distance(population)

    if model_params != None:
        n_params = 1 # need to determine the maximum number of possible parameters so that we can scale

    return 1/(1 + np.exp(-beta*fitness_input)) # * 1/diversity * n_params


In [15]:
def check_parent_similarity(parents):
    return np.any(np.sum(np.diff(parents, axis = 0), axis = 1) == 0)


In [16]:
check_parent_similarity(np.array([[1,0,0,1], [1,1,1,1]]))

False

In [18]:
print(pop.shape)
print(crossover(pop).shape)

(100, 44)
(100, 44)


In [43]:
def get_params(phenotype, parameters):

    params = []

    layers = list(parameters.keys())

    param_count = 0

    for i in range(len(layers)):

        for param_type in parameters[layers[i]].keys():
            
            params.append(parameters[layers[i]][param_type][phenotype[param_count]])
            
            param_count += 1
    
    return params


In [44]:
new_params = get_params(pheno, params)
print(new_params)

[8, 1, 16, 'same', 'relu', 'max', 2, 'same', 'spatial2D', 0.3, 'global']


In [22]:
def prep_data(sample_size = None, seed = None):
    
    X = np.load('/Users/Donovan/Documents/Masters/masters-ed02/coding/model_base_cnn/train/X.npy')
    Y = np.load('/Users/Donovan/Documents/Masters/masters-ed02/coding/model_base_cnn/train/Y.npy')

    # Subsample
    if sample_size != None:
        g_sample = int(np.floor(sample_size/2))
        n_sample = sample_size - g_sample

        g_i = np.random.randint(0, int(X.shape[0]/2) - 1, g_sample)
        n_i = np.random.randint(int(X.shape[0]/2), X.shape[0] - 1, n_sample)

        X = np.concatenate([X[g_i], X[n_i]])
        Y = np.concatenate([Y[g_i], Y[n_i]])
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.40, random_state=seed, shuffle = True)
        X_test, X_val, Y_test, Y_val = train_test_split(X_val, Y_val, test_size=0.50, random_state=seed, shuffle = True)

    else:
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.40, random_state=seed, shuffle = True)
        X_test, X_val, Y_test, Y_val = train_test_split(X_val, Y_val, test_size=0.50, random_state=seed, shuffle = True)

    return  {'X_train':X_train, 'X_val':X_val, 'X_test':X_test, 'Y_train':Y_train, 'Y_val':Y_val, 'Y_test':Y_test}

In [23]:
data_dict = prep_data(sample_size = 1000, seed = 42)

print ('X_train:',data_dict['X_train'].shape)
print ('Y_train:',data_dict['Y_train'].shape)
print ()
print ('X_val:',data_dict['X_val'].shape)
print ('Y_val:',data_dict['Y_val'].shape)
print ()
print ('X_val:',data_dict['X_test'].shape)
print ('Y_val:',data_dict['Y_test'].shape)

X_train: (600, 128, 188, 1)
Y_train: (600, 2)

X_val: (200, 128, 188, 1)
Y_val: (200, 2)

X_val: (200, 128, 188, 1)
Y_val: (200, 2)


In [24]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [25]:
def train_model(parameters, X_train, X_val, Y_train, Y_val, verbose = 0):

    model = Sequential()

    model.add(Conv2D(input_shape = X_train.shape[1:], kernel_size = parameters[1], filters = parameters[2], padding = parameters[3],
                        activation = parameters[4]))

    if parameters[8] == 'dropout':
        model.add(Dropout(rate = parameters[9]))

    else:
        model.add(SpatialDropout2D(rate = parameters[9]))

    if parameters[5] == 'max':
        model.add(MaxPool2D(pool_size=parameters[6], padding=parameters[7]))

    else:
        model.add(AveragePooling2D(pool_size=parameters[6], padding=parameters[7]))

    if parameters[10] == 'dense':
        model.add(Flatten())
        model.add(Dense(100, activation = parameters[4]))
        model.add(Dropout(rate = parameters[9]))


    else:
        model.add(GlobalAveragePooling2D())
        model.add(Flatten())
    
    model.add(Dense(2, activation = 'softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', f1_m, precision_m, recall_m])

    history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
            batch_size=parameters[0],
            epochs=10,
            verbose=verbose, 
            class_weight={0:1.,1:1.})

    return model, history

In [26]:
model, history = train_model(new_params, data_dict['X_train'], data_dict['X_val'], data_dict['Y_train'], data_dict['Y_val'], verbose=2)

2022-11-27 19:03:16.957267: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-27 19:03:16.959011: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/10


2022-11-27 19:03:17.428313: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-27 19:03:18.268470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


75/75 - 2s - loss: 0.6932 - acc: 0.4983 - f1_m: 0.4983 - precision_m: 0.4983 - recall_m: 0.4983 - val_loss: 0.6941 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 2s/epoch - 29ms/step
Epoch 2/10


2022-11-27 19:03:19.444787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


75/75 - 1s - loss: 0.6928 - acc: 0.5150 - f1_m: 0.5150 - precision_m: 0.5150 - recall_m: 0.5150 - val_loss: 0.6944 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 824ms/epoch - 11ms/step
Epoch 3/10
75/75 - 1s - loss: 0.6924 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6946 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 809ms/epoch - 11ms/step
Epoch 4/10
75/75 - 1s - loss: 0.6930 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6955 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 814ms/epoch - 11ms/step
Epoch 5/10
75/75 - 1s - loss: 0.6931 - acc: 0.5133 - f1_m: 0.5133 - precision_m: 0.5133 - recall_m: 0.5133 - val_loss: 0.6953 - val_acc: 0.4550 - val_f1_m: 0.4550 - val_precision_m: 0.4550 - val_recall_m: 0.4550 - 808ms/epoch - 11ms/step
Epoch 6/10
75/75 - 1s - loss: 0.6928 - acc: 0.5133 - f1_m: 

In [27]:
fitness_val = fitness(np.mean(np.diff(history.history['val_f1_m'])))
print(fitness_val)

0.5


In [52]:
def selection(population, data_dict, parameter_set, gene_length, k = 2, selection_probability = 0.9, seed = None, verbose = 0):

    population_size = population.shape[0]

    individual_length = population.shape[1]

    selected_population_size = 0

    if seed != None:
        np.random.seed(seed)

    while selected_population_size < population_size:

        sub_population = population[np.random.choice(np.arange(0, population_size, 1), k, replace = False)]

        scores = []

        iteration = 1

        for individual in sub_population:

            # train model outside of this function then provide fitness scores as parameter

            phenotype = decoder(individual, gene_length)

            params = get_params(phenotype, parameter_set)

            model, history = train_model(params, data_dict['X_train'], data_dict['X_val'], data_dict['Y_train'], data_dict['Y_val'], verbose = verbose)

            scores.append(fitness(np.mean(np.diff(history.history['val_f1_m']))))

            iteration += 1

        
        rank_index = np.argsort(scores)

        rank_scores = np.array(scores)[rank_index]

        ranked = sub_population[rank_index]

        p_array = np.concatenate(([selection_probability], selection_probability*((1 - selection_probability)**rank_scores)))

        p_array = np.concatenate((p_array[p_array < 1], np.array([1 - np.sum(p_array[p_array < 1])])))

        p_array = np.concatenate((p_array[0:len(ranked) - 1], [np.sum(p_array[len(ranked) - 1:])]))[rank_index]

        selected_index = np.random.choice(rank_index, p = p_array)

        selected = ranked[selected_index]

        fitness_val = rank_scores[selected_index]

        if selected_population_size == 0:
            
            selected_population = selected

            selected_fitness = np.array([fitness_val])

            selected_population_size = 1

        else:

            selected_population = np.vstack((selected_population, selected))

            selected_fitness = np.concatenate((selected_fitness, np.array([fitness_val])))

            selected_population_size = selected_population.shape[0]

    return selected_population, selected_fitness


In [ ]:
selection(pop[-2:], data_dict, params, 4)

In [58]:
def crossover(selected_population, gene_length, n_parents = 2, crossover_probability = 1.0, seed = None, verbose = 0):

    population_size = selected_population.shape[0]

    individual_length = selected_population.shape[1]

    size_tuple = (n_parents, individual_length)

    offspring_population_size = 0

    while offspring_population_size < population_size:

        similar_parents = False

        while not similar_parents:

            parent_index = np.random.choice(np.arange(0, population_size, 1), n_parents, replace = False)

            parents = selected_population[parent_index]

            similar_parents = check_parent_similarity(parents)

        if seed != None:
            np.random.seed(42)

        r = np.random.random()

        if crossover_probability < 1.0 and crossover_probability > 0.0 and r > crossover_probability:

            continue

        else:

            offspring = np.zeros(size_tuple)

            for i in range(n_parents):

                valid_individual = False

                while not valid_individual:

                    random_matrix = np.random.random(size_tuple)

                    normalised_random_matrix = random_matrix/random_matrix.sum(axis = 0)

                    max_index_j = np.argmax(normalised_random_matrix, axis = 0)

                    offspring_v = parents[max_index_j, range(individual_length)]

                    phenotype = decoder(offspring_v, gene_length)

                    valid_individual = validity_check(phenotype)
                    
                offspring[i] = offspring_v

        if offspring_population_size == 0:

            offspring_population = offspring

        else:

            offspring_population = np.vstack((offspring_population, offspring))

        offspring_population_size = offspring_population.shape[0]

    return offspring_population

In [51]:
def mutation(offspring, mutation_probability = 0.05):

    r = np.random.random((offspring.shape))

    mutated_offspring = np.copy(offspring)

    mutate_index = (r < mutation_probability)

    mutated_offspring[mutate_index] = np.logical_not(mutated_offspring[mutate_index])

    return mutated_offspring

In [55]:
def genetic_algorithm(parameters, data_dict, generations = 10, n_parents = 2, verbose = 0):

    print('Getting state space parameters....')

    individual_length, bit_length, parameter_list_lengths = get_state_space(parameters=parameters)

    print(f'Chromosome length = {individual_length}; Gene length = {bit_length}; Possible number of choices per parameter = {parameter_list_lengths}')

    print(20*'_')

    print('Generating population....')

    population = generate_population(length = individual_length, parameter_len = parameter_list_lengths)

    population_size = population.shape[0]

    print(f'Population size = {population_size}')

    generation_history = {}

    for generation in range(generations):

        g = 1

        print(20*'_')

        print(f'_Generation {g} of {generations}.')

        selected_population, selected_fitness = selection(population, data_dict, parameters, bit_length)

        offspring_population = crossover(selected_population)

        offspring_population = mutation(offspring_population)

        offspring_fitness = []

        for individual in offspring_population:

            phenotype = decoder(individual, bit_length)

            params = get_params(phenotype, parameters)

            model, history = train_model(params, data_dict['X_train'], data_dict['X_val'], data_dict['Y_train'], data_dict['Y_val'], verbose = verbose)

            offspring_fitness.append(fitness(np.mean(np.diff(history.history['val_f1_m']))))

        total_pool = np.vstack((selected_population, offspring_population))

        total_fitness = np.concatenate((selected_fitness, np.array(offspring_fitness)))

        new_population_index = np.argsort(total_fitness)[::-1][:population_size]

        population = total_pool[new_population_index]

        generation_history[str(g)]['population'] = population

        generation_history[str(g)]['fitness'] = total_fitness[new_population_index]
            
        g += 1

        clear_output(wait=True)

    return generation_history


In [56]:
genetic_algorithm(params, data_dict, verbose = 1)

Getting state space parameters....
Chromosome length = 44; Gene length = 4; Possible number of choices per parameter = [3 4 4 2 4 2 4 2 2 9 2]
____________________
Generating population....
Population size = 10
____________________
_Generation 1 of 10.


2022-11-28 06:55:37.042799: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-28 06:55:39.288425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-28 06:55:51.614480: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-28 06:55:53.152461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-28 06:56:02.270702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-28 06:56:03.791967: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-28 06:56:14.019983: I tensorflow/core/grappler/optimizers/cust

ValueError: invalid literal for int() with base 2: '0.00.00.00.0'

In [107]:
generate_population(length = 44, parameter_len = np.array([3, 4, 4, 2, 4, 2, 4, 2, 2, 9, 2]))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 

In [109]:
decoder(chromosome = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,  0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1]), 
gene_length = 4)

array([0, 0, 1, 1, 1, 1, 3, 0, 1, 3, 1])

In [110]:
selection(generate_population(length = 44, parameter_len = np.array([3, 4, 4, 2, 4, 2, 4, 2, 2, 9, 2])), phenotype=decoder(chromosome = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,  0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1]), 
gene_length = 4))

In [55]:
np.vstack((np.array([[1,2,3],[1,2,3]]),np.array([1,2,3])))

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])